# Practice for GPU and Use of TF.SLIM

<p> For GPU simply specify the device. As it can be observed below there is the option to use 
GPU or CPU</p>

In [1]:
import tensorflow.contrib.slim as slim

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
import tensorflow as tf
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[ 22.  28.]
 [ 49.  64.]]


In [4]:
# Creates a graph.
with tf.device('/cpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[ 22.  28.]
 [ 49.  64.]]


In [5]:
# Creates a graph.
with tf.device('/gpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))


[[ 22.  28.]
 [ 49.  64.]]


### Use  allow_soft_placement 
 <p> Should the specified device not be available use the allow_soft_placement to ensure
 it defaults to any available node  </p>

In [6]:
# Creates a graph.
with tf.device('/cpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)
# Creates a session with allow_soft_placement and log_device_placement set
# to True.
sess = tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[ 22.  28.]
 [ 49.  64.]]


### Using the Slim package

In [3]:
# defining variable
weights = slim.variable('weights',
                             shape=[10, 10, 3 , 3],
                             initializer=tf.truncated_normal_initializer(stddev=0.1),
                             regularizer=slim.l2_regularizer(0.05),
                             #reuse=True,
                             device='/gpu:0')

In [4]:
print(weights.name)

weights:0


In [5]:
# Model and regular variables in Slim
#### Make model variables in slim package 
# Model Variables
weights = slim.model_variable('weight',
                              shape=[10, 10, 3 , 3],
                              initializer=tf.truncated_normal_initializer(stddev=0.1),
                              regularizer=slim.l2_regularizer(0.05),
                              device='/gpu:0')
model_variables = slim.get_model_variables()

# Regular variables
my_var = slim.variable('my_var',
                       shape=[30,30,3,3],
                       #initializer=tf.zeros_initializer()
                      )
regular_variables_and_model_variables = slim.get_variables()

In [6]:
type(model_variables) # model variables

list

In [7]:
model_variables   # regular variables

[<tf.Variable 'weight:0' shape=(10, 10, 3, 3) dtype=float32_ref>]

In [8]:
regular_variables_and_model_variables  # regular and model variables

[<tf.Variable 'weights:0' shape=(10, 10, 3, 3) dtype=float32_ref>,
 <tf.Variable 'weight:0' shape=(10, 10, 3, 3) dtype=float32_ref>,
 <tf.Variable 'my_var:0' shape=(30, 30, 3, 3) dtype=float32_ref>]

## Build convolution layer

### Simple Convolution in tf.slim

In [9]:
net = slim.conv2d(my_var, 128, [3, 3], scope='conv1_1') # convolution with slim 

In [10]:
net # see the convolution layer

<tf.Tensor 'conv1_1/Relu:0' shape=(30, 30, 3, 128) dtype=float32>

In [11]:
my_var # my variable to be used as input

<tf.Variable 'my_var:0' shape=(30, 30, 3, 3) dtype=float32_ref>

In [12]:
sess = tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True)) # create a session

sess.run(tf.global_variables_initializer()) # initialise global variables

with tf.device('/gpu:0'):
    my_var_= np.arange(8100).reshape(30,30,3,3) # make variable to feed to my_var
    
    convolution=sess.run(net, feed_dict={my_var: my_var_})# run convolution net
    
    print("Convolution output shape  is ", convolution.shape)
    print("\n")
    print(convolution[1])



('Convolution output shape  is ', (30, 30, 3, 128))


[[[  28.93692398   10.7195673     0.         ...,   30.05235672    0.            0.        ]
  [  46.07234955    7.70915699    0.         ...,    0.            0.            0.        ]
  [  16.22345352    0.            0.         ...,    0.            0.            0.        ]]

 [[  15.06083679   57.84450912    0.         ...,   43.14462662
     18.08991241    0.        ]
  [  42.63002777   78.63479614    0.         ...,    0.           49.88648605
      0.        ]
  [  25.13093185   57.33727646    0.         ...,    0.           39.67684937
      0.        ]]

 [[  15.48501587   59.74223328    0.         ...,   44.54095459
     18.81566811    0.        ]
  [  43.94680786   81.18474579    0.         ...,    0.           51.65623474
      0.        ]
  [  25.93070221   59.19885254    0.         ...,    0.           41.04518127
      0.        ]]

 ..., 
 [[  26.08940315  107.18469238    0.         ...,   79.44909668
     36.959522

### Meta Operation : repeat and stack

for multiple convolution loop, instead of this:

<p>net = slim.conv2d(net, 256, [3, 3], scope='conv3_1')</p>
<p>net = slim.conv2d(net, 256, [3, 3], scope='conv3_2')</p>
<p>net = slim.conv2d(net, 256, [3, 3], scope='conv3_3')</p>
<p>net = slim.max_pool2d(net, [2, 2], scope='pool2')</p>
<p>Use this :</p>
<p>
net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')</p>



In [17]:
net = slim.repeat(net, 3, slim.conv2d, 128, [3, 3], scope='conv3')
net = slim.max_pool2d(net, [2, 2], scope='pool2')


In [18]:
net

<tf.Tensor 'pool2/MaxPool:0' shape=(30, 15, 1, 128) dtype=float32>

In [19]:
sess.run(tf.global_variables_initializer()) # initialise global variables


with tf.device('/gpu:0'):
    my_var_= np.arange(8100).reshape(30,30,3,3) # make variable to feed to my_var
    
    convolution=sess.run(net, feed_dict={my_var: my_var_})# run convolution net
    
    print("repeated convolution and pool layer output shape  is ", convolution.shape)
    print("\n")
    print(convolution[1])

('repeated convolution and pool layer output shape  is ', (30, 15, 1, 128))


[[[ 14.90907288  15.58975983  21.07088661 ...,  14.6812706   20.05348969
     0.        ]]

 [[ 15.48785782  20.05633354   8.90880394 ...,  17.70003319  11.46362972
     0.        ]]

 [[ 14.35120773  22.78635597   8.12516022 ...,  22.27364731  12.7917366
     0.        ]]

 ..., 
 [[ 22.40684319  35.17812347  12.29501343 ...,  34.45668793  19.77536201
     0.        ]]

 [[ 26.25650787  36.48138046  11.01838493 ...,  38.84376526  19.45845985
     0.        ]]

 [[ 27.58261681  25.97477531  12.69604969 ...,  16.76501083  11.35532188
     0.        ]]]


# stack  operation
<p>slim.stack operator allows a caller to repeatedly apply the same operation </p>
<p>with different arguments to create a stack or tower of layers. slim.stack also </p>
</p>creates a new tf.variable_scope for each operation created. For example, a </p>
simple way to create a Multi-Layer Perceptron (MLP):

In [20]:
fully_connected_stack = slim.stack(net, slim.fully_connected, [32, 64, 128], scope='fc')


In [21]:
net

<tf.Tensor 'pool2/MaxPool:0' shape=(30, 15, 1, 128) dtype=float32>

In [25]:
sess.run(tf.global_variables_initializer()) # initialise global variables


with tf.device('/gpu:0'):
    my_var_= np.arange(8100).reshape(30,30,3,3) # make variable to feed to my_var
    
    convolution=sess.run(net, feed_dict={my_var: my_var_})# run convolution net
    #print(convolution.shape)
    full_connected = sess.run(fully_connected_stack, feed_dict={net:convolution})
    
    print("repeated convolution , pool layer   and stacked full connected output shape  is ", full_connected.shape)
    print("\n")
    print(full_connected[1])

(30, 15, 1, 128)
('repeated convolution , pool layer   and stacked full connected output shape  is ', (30, 15, 1, 128))


[[[  4.23216581   2.04975176   0.         ...,   4.95490932   0.73523045
     0.        ]]

 [[  2.58513737   1.35162556   2.74485016 ...,   5.63072109   0.           0.        ]]

 [[  2.59122324   1.58042955   1.35051274 ...,   7.29484463   0.06100845
     0.        ]]

 ..., 
 [[  4.20043468   2.72412586   2.04240131 ...,  11.6093235    0.           0.        ]]

 [[  6.1571722    4.36448002   5.0669136  ...,  11.89010239   0.           0.        ]]

 [[  5.46519947   7.07594013   5.87183571 ...,  11.75918865   1.83790112
     0.        ]]]


### Consider this stacked convolution
 Verbose way:
<p>net = slim.fully_connected(net, 32, scope='fc/fc_1')</p>
<p>net = slim.fully_connected(net, 64, scope='fc/fc_2')</p>
<p>net = slim.fully_connected(net, 128, scope='fc/fc_3')</p>



In [23]:
# Equivalent, TF-Slim way using slim.stack:
stacked_convolution = slim.stack(net, slim.conv2d, [(32, [3, 3]), (32, [1, 1]), (64, [3, 3]), (64, [1, 1])], scope='core')

In [27]:
sess.run(tf.global_variables_initializer()) # initialise global variables


with tf.device('/gpu:0'):
    my_var_= np.arange(8100).reshape(30,30,3,3) # make variable to feed to my_var
    
    convolution=sess.run(net, feed_dict={my_var: my_var_})# run convolution net
    stacked_connected = sess.run(stacked_convolution, feed_dict={net:convolution})
    
    print("repeated convolution , pool layer   and stacked full connected output shape  is ", stacked_connected.shape)
    print("\n")
    print(stacked_connected[1][1])

('repeated convolution , pool layer   and stacked full connected output shape  is ', (30, 15, 1, 64))


[[ 0.          0.          0.7477169   0.13506943  0.9713673   0.
   5.1420927   1.4078269   1.6295898   0.15620512  2.0037446   0.          0.
   0.          0.          1.18696272  0.          0.          0.          0.
   1.11823201  0.          0.          0.72954273  0.          0.
   1.33379316  0.93567711  0.          1.49034071  0.43626654  0.38108742
   0.          0.          2.00853372  1.92772484  0.68287849  0.04531655
   2.12834167  0.          0.26476377  0.          0.80931449  1.23653197
   0.          2.44665647  0.          0.          0.          0.84878552
   0.          0.51374352  0.          0.11831701  1.56720352  0.23275542
   0.          2.39716816  0.69503832  3.70868254  0.          0.10966003
   0.          2.09749699]]


### Scopes



Using scope to reduce the amount of operation being conducted.

In [13]:
# define fequently use variable in a scope: padding and weight
with slim.arg_scope([slim.conv2d], padding='SAME',
                      weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                      weights_regularizer=slim.l2_regularizer(0.0005)
                   ):
    net = slim.conv2d(my_var, 64, [3, 3], scope='conv1')
    net = slim.conv2d(net, 128, [3, 3], padding='VALID', scope='conv2')
    net = slim.conv2d(net, 256, [3, 3], scope='conv3')

In [14]:
sess.run(tf.global_variables_initializer()) # initialise global variables


with tf.device('/gpu:0'):
    my_var_= np.arange(8100).reshape(30,30,3,3) # make variable to feed to my_var
    
    #convolution=sess.run(net, feed_dict={my_var: my_var_})# run convolution net
    scope_convolution = sess.run(net, feed_dict={my_var:my_var_})
    
    print("scope convolution__", scope_convolution.shape)
    print("\n")
    print(scope_convolution[1])

('scope convolution__', (30, 28, 1, 256))


[[[ 0.          0.          0.30117792 ...,  0.          0.          0.        ]]

 [[ 0.          0.          0.46823403 ...,  0.          0.          0.        ]]

 [[ 0.          0.          0.49150071 ...,  0.          0.          0.        ]]

 ..., 
 [[ 0.          0.          0.81940919 ...,  0.          0.          0.        ]]

 [[ 0.          0.          0.62286884 ...,  0.          0.          0.01223324]]

 [[ 0.01849812  0.          0.43253469 ...,  0.18174121  0.          0.01509733]]]
